In [10]:
!pip install pydrive
!pip install GoogleCredential
!pip install auth
#!pip install -q time
#!pip install -q glob
!pip install -q keras
!pip install -q IPython
!pip install -q matplotlib
!pip install -q numpy
!pip install -q pandas
!pip install -q plotly
!pip install -q tensorflow
!pip install -q seaborn
!pip install -q scikit-learn
!pip install tqdm
!pip install -q os-sys
!pip install -q librosa
!pip install -q resampy
!pip install -q sounddevice
!pip install pydub


ERROR: Could not find a version that satisfies the requirement GoogleCredential (from versions: none)
ERROR: No matching distribution found for GoogleCredential
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


In [19]:
import IPython
from pydub import AudioSegment
import numpy as np
from scipy.io import wavfile
from plotly.offline import init_notebook_mode
import scipy.signal
import matplotlib.pyplot as plt
import librosa
import plotly.graph_objs as go
import plotly
import os
import time
from datetime import timedelta as td
import glob
from scipy.io import wavfile
from tqdm import tqdm

In [13]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [14]:
problem =[300, 305, 306, 308, 315, 316, 343, 354, 362, 375, 378, 381, 382, 385, 387, 388, 390, 392, 393, 395, 408, 413, 421, 438, 473, 476, 479, 490, 492]

In [15]:
# NOISE REMOVAL CODE from https://timsainburg.com/noise-reduction-python.html
def _stft(y, n_fft, hop_length, win_length):
    return librosa.stft(y=y, n_fft=n_fft, hop_length=hop_length, win_length=win_length)


def _istft(y, hop_length, win_length):
    return librosa.istft(y, hop_length, win_length)


def _amp_to_db(x):
    return librosa.core.amplitude_to_db(x, ref=1.0, amin=1e-20, top_db=80.0)


def _db_to_amp(x,):
    return librosa.core.db_to_amplitude(x, ref=1.0)


def plot_spectrogram(signal, title):
    fig, ax = plt.subplots(figsize=(20, 4))
    cax = ax.matshow(
        signal,
        origin="lower",
        aspect="auto",
        cmap=plt.cm.seismic,
        vmin=-1 * np.max(np.abs(signal)),
        vmax=np.max(np.abs(signal)),
    )
    fig.colorbar(cax)
    ax.set_title(title)
    plt.tight_layout()
    plt.show()


def plot_statistics_and_filter(
    mean_freq_noise, std_freq_noise, noise_thresh, smoothing_filter
):
    fig, ax = plt.subplots(ncols=2, figsize=(20, 4))
    plt_mean, = ax[0].plot(mean_freq_noise, label="Mean power of noise")
    plt_std, = ax[0].plot(std_freq_noise, label="Std. power of noise")
    plt_std, = ax[0].plot(noise_thresh, label="Noise threshold (by frequency)")
    ax[0].set_title("Threshold for mask")
    ax[0].legend()
    cax = ax[1].matshow(smoothing_filter, origin="lower")
    fig.colorbar(cax)
    ax[1].set_title("Filter for smoothing Mask")
    plt.show()


def removeNoise(
    audio_clip,
    noise_clip,
    n_grad_freq=2,
    n_grad_time=4,
    n_fft=2048,
    win_length=2048,
    hop_length=512,
    n_std_thresh=1.5,
    prop_decrease=1.0,
    verbose=False,
    visual=False,
):
    """Remove noise from audio based upon a clip containing only noise

    Args:
        audio_clip (array): The first parameter.
        noise_clip (array): The second parameter.
        n_grad_freq (int): how many frequency channels to smooth over with the mask.
        n_grad_time (int): how many time channels to smooth over with the mask.
        n_fft (int): number audio of frames between STFT columns.
        win_length (int): Each frame of audio is windowed by `window()`. The window will be of length `win_length` and then padded with zeros to match `n_fft`..
        hop_length (int):number audio of frames between STFT columns.
        n_std_thresh (int): how many standard deviations louder than the mean dB of the noise (at each frequency level) to be considered signal
        prop_decrease (float): To what extent should you decrease noise (1 = all, 0 = none)
        visual (bool): Whether to plot the steps of the algorithm

    Returns:
        array: The recovered signal with noise subtracted

    """
    if verbose:
        start = time.time()
    # STFT over noise
    noise_stft = _stft(noise_clip, n_fft, hop_length, win_length)
    noise_stft_db = _amp_to_db(np.abs(noise_stft))  # convert to dB
    # Calculate statistics over noise
    mean_freq_noise = np.mean(noise_stft_db, axis=1)
    std_freq_noise = np.std(noise_stft_db, axis=1)
    noise_thresh = mean_freq_noise + std_freq_noise * n_std_thresh
    if verbose:
        print("STFT on noise:", td(seconds=time.time() - start))
        start = time.time()
    # STFT over signal
    if verbose:
        start = time.time()
    sig_stft = _stft(audio_clip, n_fft, hop_length, win_length)
    sig_stft_db = _amp_to_db(np.abs(sig_stft))
    if verbose:
        print("STFT on signal:", td(seconds=time.time() - start))
        start = time.time()
    # Calculate value to mask dB to
    mask_gain_dB = np.min(_amp_to_db(np.abs(sig_stft)))
    print(noise_thresh, mask_gain_dB)
    # Create a smoothing filter for the mask in time and frequency
    smoothing_filter = np.outer(
        np.concatenate(
            [
                np.linspace(0, 1, n_grad_freq + 1, endpoint=False),
                np.linspace(1, 0, n_grad_freq + 2),
            ]
        )[1:-1],
        np.concatenate(
            [
                np.linspace(0, 1, n_grad_time + 1, endpoint=False),
                np.linspace(1, 0, n_grad_time + 2),
            ]
        )[1:-1],
    )
    smoothing_filter = smoothing_filter / np.sum(smoothing_filter)
    # calculate the threshold for each frequency/time bin
    db_thresh = np.repeat(
        np.reshape(noise_thresh, [1, len(mean_freq_noise)]),
        np.shape(sig_stft_db)[1],
        axis=0,
    ).T
    # mask if the signal is above the threshold
    sig_mask = sig_stft_db < db_thresh
    if verbose:
        print("Masking:", td(seconds=time.time() - start))
        start = time.time()
    # convolve the mask with a smoothing filter
    sig_mask = scipy.signal.fftconvolve(sig_mask, smoothing_filter, mode="same")
    sig_mask = sig_mask * prop_decrease
    if verbose:
        print("Mask convolution:", td(seconds=time.time() - start))
        start = time.time()
    # mask the signal
    sig_stft_db_masked = (
        sig_stft_db * (1 - sig_mask)
        + np.ones(np.shape(mask_gain_dB)) * mask_gain_dB * sig_mask
    )  # mask real
    sig_imag_masked = np.imag(sig_stft) * (1 - sig_mask)
    sig_stft_amp = (_db_to_amp(sig_stft_db_masked) * np.sign(sig_stft)) + (
        1j * sig_imag_masked
    )
    if verbose:
        print("Mask application:", td(seconds=time.time() - start))
        start = time.time()
    # recover the signal
    recovered_signal = _istft(sig_stft_amp, hop_length, win_length)
    recovered_spec = _amp_to_db(
        np.abs(_stft(recovered_signal, n_fft, hop_length, win_length))
    )
    if verbose:
        print("Signal recovery:", td(seconds=time.time() - start))
    if visual:
        plot_spectrogram(noise_stft_db, title="Noise")
    if visual:
        plot_statistics_and_filter(
            mean_freq_noise, std_freq_noise, noise_thresh, smoothing_filter
        )
    if visual:
        plot_spectrogram(sig_stft_db, title="Signal")
    if visual:
        plot_spectrogram(sig_mask, title="Mask applied")
    if visual:
        plot_spectrogram(sig_stft_db_masked, title="Masked signal")
    if visual:
        plot_spectrogram(recovered_spec, title="Recovered spectrogram")
    return recovered_signal

In [20]:


# Fixing the istft function
def _istft(y, hop_length=512, win_length=2048):
    """Wrapper for librosa.istft to handle signal reconstruction."""
    return librosa.istft(y, hop_length=hop_length, win_length=win_length)

# Other helper functions
def _stft(y, n_fft, hop_length, win_length):
    return librosa.stft(y=y, n_fft=n_fft, hop_length=hop_length, win_length=win_length)

def _amp_to_db(x):
    return librosa.amplitude_to_db(x, ref=1.0, amin=1e-20, top_db=80.0)

def _db_to_amp(x):
    return librosa.db_to_amplitude(x, ref=1.0)

def get_noise_clip(audio_clip, rate_audio, duration_sec=1):
    """Extract the first `duration_sec` seconds as the noise clip."""
    noise_samples = rate_audio * duration_sec
    return audio_clip[:noise_samples]  # Extract first `duration_sec` seconds

# Noise removal function
def removeNoise(
    audio_clip,
    noise_clip,
    n_fft=2048,
    win_length=2048,
    hop_length=512,
    n_std_thresh=1.5,
    prop_decrease=1.0,
):
    noise_stft = _stft(noise_clip, n_fft, hop_length, win_length)
    noise_stft_db = _amp_to_db(np.abs(noise_stft))
    mean_freq_noise = np.mean(noise_stft_db, axis=1)
    std_freq_noise = np.std(noise_stft_db, axis=1)
    noise_thresh = mean_freq_noise + std_freq_noise * n_std_thresh

    sig_stft = _stft(audio_clip, n_fft, hop_length, win_length)
    sig_stft_db = _amp_to_db(np.abs(sig_stft))

    mask_gain_dB = np.min(_amp_to_db(np.abs(sig_stft)))
    db_thresh = np.repeat(np.reshape(noise_thresh, [1, len(mean_freq_noise)]), np.shape(sig_stft_db)[1], axis=0).T
    sig_mask = sig_stft_db < db_thresh

    smoothing_filter = np.outer(
        np.concatenate([np.linspace(0, 1, 2, endpoint=False), np.linspace(1, 0, 2)]),
        np.concatenate([np.linspace(0, 1, 2, endpoint=False), np.linspace(1, 0, 2)]),
    )
    smoothing_filter = smoothing_filter / np.sum(smoothing_filter)
    sig_mask = scipy.signal.fftconvolve(sig_mask, smoothing_filter, mode="same")
    sig_mask = sig_mask * prop_decrease

    sig_stft_db_masked = sig_stft_db * (1 - sig_mask) + mask_gain_dB * sig_mask
    sig_stft_amp = (_db_to_amp(sig_stft_db_masked) * np.sign(sig_stft)) + (1j * np.imag(sig_stft) * (1 - sig_mask))

    # Reconstruct the signal
    recovered_signal = _istft(sig_stft_amp, hop_length=hop_length, win_length=win_length)
    return recovered_signal

# Input and output directories
input_dir = "/content/drive/MyDrive/Colab Notebooks/audio-step1-diacWoz"
output_dir = "/content/drive/MyDrive/Colab Notebooks/audio-step2-noise-removed"

os.makedirs(output_dir, exist_ok=True)

# Process all .wav files in the input directory
for file_path in tqdm(glob.glob(os.path.join(input_dir, "*.wav"))):
    if int(os.path.basename(file_path)[0:3]) in problem:
        continue  # Skip files with prefixes in the 'problem' list

    try:
        rate_audio, audio_clip = wavfile.read(file_path)
        audio_clip = audio_clip / (2**15)  # Normalize audio to [-1, 1]

        # Extract noise dynamically from the first 1 second of the file
        noise_clip = get_noise_clip(audio_clip, rate_audio, duration_sec=1)

        # Apply noise reduction
        noise_reduced_clip = removeNoise(audio_clip=audio_clip, noise_clip=noise_clip)

        # Revert normalization for saving
        noise_reduced_clip = np.int16(noise_reduced_clip * (2**15))

        # Generate new filename
        original_filename = os.path.basename(file_path)
        file_id = original_filename[:3]
        new_filename = f"{file_id}_noise_removed.wav"

        # Save the noise-reduced file to the output directory
        output_file_path = os.path.join(output_dir, new_filename)
        wavfile.write(output_file_path, rate_audio, noise_reduced_clip)
        print(f"Processed and saved: {output_file_path}")

    except Exception as e:
        print(f"Error processing {file_path}: {e}")


  0%|          | 0/70 [00:00<?, ?it/s]<ipython-input-20-9f7fc662f4f7>:71: WavFileWarning:

Reached EOF prematurely; finished at 26364844 bytes, expected 210918444 bytes from header.

  1%|▏         | 1/70 [00:12<13:55, 12.11s/it]

Processed and saved: /content/drive/MyDrive/Colab Notebooks/audio-step2-noise-removed/301_noise_removed.wav


<ipython-input-20-9f7fc662f4f7>:71: WavFileWarning:

Reached EOF prematurely; finished at 24281644 bytes, expected 194252844 bytes from header.

  3%|▎         | 2/70 [00:23<12:57, 11.44s/it]

Processed and saved: /content/drive/MyDrive/Colab Notebooks/audio-step2-noise-removed/302_noise_removed.wav


<ipython-input-20-9f7fc662f4f7>:71: WavFileWarning:

Reached EOF prematurely; finished at 31529644 bytes, expected 252236844 bytes from header.

  4%|▍         | 3/70 [00:37<14:14, 12.75s/it]

...

Processed and saved: /content/drive/MyDrive/Colab Notebooks/audio-step2-noise-removed/340_noise_removed.wav


<ipython-input-20-9f7fc662f4f7>:71: WavFileWarning:

Reached EOF prematurely; finished at 27612844 bytes, expected 220902444 bytes from header.

 99%|█████████▊| 69/70 [14:41<00:13, 13.63s/it]

Processed and saved: /content/drive/MyDrive/Colab Notebooks/audio-step2-noise-removed/339_noise_removed.wav


<ipython-input-20-9f7fc662f4f7>:71: WavFileWarning:

Reached EOF prematurely; finished at 59878444 bytes, expected 479027244 bytes from header.

100%|██████████| 70/70 [15:13<00:00, 13.05s/it]

Processed and saved: /content/drive/MyDrive/Colab Notebooks/audio-step2-noise-removed/337_noise_removed.wav
